In [ ]:
# Imported Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from sklearn.cluster import KMeans
from math import ceil

In [ ]:
#Enter Source Name
source_name=input('Enter Source name (Format="V1139_cyg"): ')

In [ ]:
# Load YSO candidate list
csv_path = f"selected_ysos_{source_name}.csv" 
ysos_df = pd.read_csv(csv_path)

# Compute median center for RA and DEC
ra_center_deg = ysos_df['RA_deg'].median()
dec_center_deg = ysos_df['DEC_deg'].median()
coord_center = SkyCoord(ra=ra_center_deg*u.deg, dec=dec_center_deg*u.deg)

print(f" Median center (in Deg): RA={ra_center_deg}, DEC={dec_center_deg}")


In [ ]:
# Convert all coordinates to SkyCoord
ysos_df['coord'] = [SkyCoord(ra=ra*u.deg, dec=dec*u.deg)
                    for ra, dec in zip(ysos_df['RA_deg'], ysos_df['DEC_deg'])]
# compute separation from center
ysos_df['offset_arcmin'] = [c.separation(coord_center).arcminute for c in ysos_df['coord']]

# Filter objects between 0.25 and 6 arcmin
ysos_df = ysos_df[(ysos_df['offset_arcmin'] > 0.25) & (ysos_df['offset_arcmin'] <= 6)]


In [ ]:
# Group objects by brightness in 0.5 magnitude bins
ysos_df['Jmag_bin'] = ysos_df['Jmag'].apply(lambda x: round(x * 2) / 2)
magnitude_groups = ysos_df.groupby('Jmag_bin')


In [ ]:
# Filteration by offset 
filtered_groups = {
    mag: group[(group['offset_arcmin'] > 0.25) & (group['offset_arcmin'] <= 6)].copy()
    for mag, group in magnitude_groups if not group.empty
}

# Divide each group into chunks of max 8
sublists = []
for mag, group in filtered_groups.items():
    group = group.sort_values('offset_arcmin')
    chunks = [group.iloc[i:i+8].copy() for i in range(0, len(group), 8)]
    sublists.extend(chunks)


In [ ]:
# Collision check if all targets are ≥ 0.3 arcmin apart
def collision_check(sublist, min_sep=0.3):
    valid_targets = []
    for _, row in sublist.iterrows():
        this_coord = row['coord']
        if all(this_coord.separation(other['coord']).arcminute >= min_sep for other in valid_targets):
            valid_targets.append(row)
    return pd.DataFrame(valid_targets)

# Apply to all sublists
collision_free_sublists = [collision_check(sublist) for sublist in sublists]


In [ ]:
# KMeans clustering if group has > 8 members
final_sublists = []

for group in collision_free_sublists:
    if len(group) > 8:
        coords_array = np.array([[c.ra.degree, c.dec.degree] for c in group['coord']])
        k = ceil(len(group) / 8)
        kmeans = KMeans(n_clusters=k, random_state=42).fit(coords_array)
        group['cluster'] = kmeans.labels_
        for label in range(k):
            cluster_group = group[group['cluster'] == label].drop(columns='cluster')
            final_sublists.append(cluster_group)
    else:
        final_sublists.append(group)


In [ ]:
# Format and save output
output = []
for i, group in enumerate(final_sublists, 1):
    for _, row in group.iterrows():
        coord = row['coord']
        ra_hms, dec_dms = coord.to_string('hmsdms').split()
        output.append({
            'Group': i,
            'GAIA_Source_ID': row['GAIA_Source_ID'],
            'RA_HMS': ra_hms,
            'DEC_DMS': dec_dms,
            'RA_deg': coord.ra.degree,
            'DEC_deg': coord.dec.degree,
            'Jmag': row['Jmag'],
            'Offset_arcmin': row['offset_arcmin']
        })

df_out = pd.DataFrame(output)
df_out.to_csv(f"grouped_ysos_{source_name}.csv", index=False)

print(f" Done! Total groups: {len(final_sublists)}. Results saved to 'grouped_ysos_{source_name}.csv'.")


In [ ]:
# Create output folder for previews
preview_dir = f"group_plot_previews_{source_name}"
os.makedirs(preview_dir, exist_ok=True)
colors = plt.cm.tab10.colors  # 10-color palette

# Plot each group with RA/DEC offsets
for i, group in enumerate(final_sublists, 1):
    fig, ax = plt.subplots(figsize=(8, 8))
    group_coords = SkyCoord(ra=group['RA_deg'].values*u.deg, dec=group['DEC_deg'].values*u.deg)
    center_coord = SkyCoord(ra=ra_center_deg*u.deg, dec=dec_center_deg*u.deg)

    delta_ra = (group_coords.ra - center_coord.ra).arcmin
    delta_dec = (group_coords.dec - center_coord.dec).arcmin

    plt.scatter(delta_ra, delta_dec, label=f'Group {i}', color=colors[(i - 1) % 10], s=50)
    plt.scatter(0, 0, c='black', marker='*', s=150, label='Center')

    x_ticks = np.linspace(-6, 6, 5)
    y_ticks = np.linspace(-6, 6, 5)
    plt.xticks(x_ticks, [f"{x:.1f}" for x in x_ticks])
    plt.yticks(y_ticks, [f"{y:.1f}" for y in y_ticks])

    plt.xlabel("ΔRA (arcmin)")
    plt.ylabel("ΔDEC (arcmin)")
    plt.title(f"Group {i} - 2MASS-K")
    plt.grid(True)
    plt.legend()
    plt.gca().invert_xaxis()
    plt.tight_layout()

    preview_path = os.path.join(preview_dir, f"group_{i}_plot_preview_{source_name}.png")
    plt.savefig(preview_path, bbox_inches='tight')
    plt.close()

print(f" Group Preview Plots saved to folder: '{preview_dir}'")
